# run_all() Function Example

The run all function in exovetter is a way to run a set of vetters on a list of tces and light curves. The function returns a dataframe with the desired vetting results and can return diagnostic plots for each tce/light curve pair. 


In [1]:
from exovetter import vetters as vet
from exovetter import utils
import lightkurve as lk
import numpy as np
import os
%load_ext autoreload
%autoreload 2

## Setup
Here we will generate a list of 5 tces and light curves to vet on from some tess objects of interest.

In [6]:
tois = ['TOI_1000.01','TOI_1001.01','TOI_1004.01', 'TOI_1007.01', 'TOI_1011.01']

tces = []
for toi in tois:
    tce = utils.get_mast_tce(toi, catalogs=['toi'])[0] # get a tce from the toi catalog info on mast for each of the input tois
    tce['snr'] = 10 # add a snr value for LPP. The code defaults to 10 but placing here to avoid warnings printing to console later
    tces.append(tce)

print('fist tce:')
display(tces[0])

fist tce:


{'period': <Quantity 2.17134839 d>,
 'epoch': <Quantity 58492.88398 d>,
 'epoch_offset': <Quantity -2400000.5 d>,
 'depth': <Quantity 0.00065689>,
 'duration': <Quantity 0.08070833 d>,
 'target': 'TOI_1000.01',
 'catalog_name': 'toi',
 'snr': 10}

Next we create 5 flattened lightcurves for each of these tces

In [7]:
lcs = []
for toi in tois: # Get each tess-spoc 30 min lightcurve for a given TOI
    lk_search = lk.search_lightcurve(target=toi, mission="TESS", author='TESS-SPOC', exptime=1800)
    all_lcs = lk_search.download_all()
    lc = all_lcs.stitch() # if there are multiple stitch them together
    lc = lc[~np.isnan(lc["flux"]) & (lc["quality"] == 0)] # remove nans/flatten
    lc = lc.flatten()
    
    lcs.append(lc) # put into a list

print('first light curve:')
display(lcs[0])

first light curve:


time,flux,flux_err,timecorr,cadenceno,centroid_col,centroid_row,sap_flux,sap_flux_err,sap_bkg,sap_bkg_err,pdcsap_flux,pdcsap_flux_err,quality,psf_centr1,psf_centr1_err,psf_centr2,psf_centr2_err,mom_centr1,mom_centr1_err,mom_centr2,mom_centr2_err,pos_corr1,pos_corr2
,,,d,,pix,pix,electron / s,electron / s,electron / s,electron / s,electron / s,electron / s,,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix
Time,float64,float64,float32,int32,float64,float64,float32,float32,float32,float32,float32,float32,int32,float64,float32,float64,float32,float64,float32,float64,float32,float32,float32
1491.661453142677,9.9996472e-01,2.2554856e-04,4.6743164e-03,12681,436.52100,755.97725,2.1254082e+04,4.5699348e+00,3.2045823e+03,1.3912508e+00,2.3087080e+04,5.2074413e+00,0,———,———,———,———,436.52100,1.5927307e-04,755.97725,1.7754840e-04,-2.1871334e-02,1.1810441e-02
1491.682286850569,9.9969400e-01,2.2460553e-04,4.6746996e-03,12682,436.52088,755.98137,2.1253826e+04,4.5507998e+00,3.0577244e+03,1.3681870e+00,2.3080686e+04,5.1856365e+00,0,———,———,———,———,436.52088,1.5853009e-04,755.98137,1.7645523e-04,-2.2363646e-02,1.7077144e-02
1491.7031205579951,9.9979858e-01,2.2382242e-04,4.6750824e-03,12683,436.52041,755.98406,2.1260943e+04,4.5349050e+00,2.9233687e+03,1.3473932e+00,2.3082957e+04,5.1675243e+00,0,———,———,———,———,436.52041,1.5791379e-04,755.98406,1.7545884e-04,-2.2318404e-02,2.0602979e-02
1491.7239542649556,1.0001779e+00,2.2303042e-04,4.6754647e-03,12684,436.52072,755.98656,2.1270973e+04,4.5188298e+00,2.7991401e+03,1.3261211e+00,2.3091574e+04,5.1492071e+00,0,———,———,———,———,436.52072,1.5716834e-04,755.98656,1.7445495e-04,-2.2171197e-02,2.4176668e-02
1491.7447879714507,1.0002446e+00,2.2222744e-04,4.6758465e-03,12685,436.52080,755.98880,2.1272453e+04,4.5025339e+00,2.6798447e+03,1.3062884e+00,2.3092973e+04,5.1306376e+00,0,———,———,———,———,436.52080,1.5642273e-04,755.98880,1.7369071e-04,-2.1976940e-02,2.7207050e-02
1491.76562167748,9.9965883e-01,2.2152253e-04,4.6762279e-03,12686,436.52090,755.99111,2.1263074e+04,4.4882245e+00,2.5745127e+03,1.2880743e+00,2.3079311e+04,5.1143322e+00,0,———,———,———,———,436.52090,1.5596587e-04,755.99111,1.7292640e-04,-2.1390729e-02,3.0201580e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1515.9738491119895,1.0001315e+00,2.1706852e-04,4.5804163e-03,13848,436.55456,755.91542,2.1180740e+04,4.3973360e+00,1.9478334e+03,1.1776276e+00,2.3086826e+04,5.0107646e+00,0,———,———,———,———,436.55456,1.5255045e-04,755.91542,1.7026313e-04,2.0531051e-02,-6.2384330e-02


## run_all()

The vet.run_all() takes a list of tces and associated light curves and runs a specified list of vetters on them. The results of each vetter is packed into a dataframe which the function returns. By default it runs every vetter except centroid which is currently being added. 

In [8]:
results = utils.run_all(tces, lcs)
results

/Users/mdallas/Desktop/exovetter/exovetter/leo.py:16: RuntimeWarning: invalid value encountered in scalar divide
  mean = np.sum(w * y) / np.sum(w)
/Users/mdallas/Desktop/exovetter/exovetter/leo.py:21: RuntimeWarning: divide by zero encountered in scalar divide
  err = 1 / np.sqrt(np.sum(w))
/Users/mdallas/Desktop/exovetter/exovetter/leo.py:16: RuntimeWarning: invalid value encountered in scalar divide
  mean = np.sum(w * y) / np.sum(w)
/Users/mdallas/Desktop/exovetter/exovetter/leo.py:21: RuntimeWarning: divide by zero encountered in scalar divide
  err = 1 / np.sqrt(np.sum(w))
/Users/mdallas/Desktop/exovetter/exovetter/leo.py:16: RuntimeWarning: invalid value encountered in scalar divide
  mean = np.sum(w * y) / np.sum(w)
/Users/mdallas/Desktop/exovetter/exovetter/leo.py:21: RuntimeWarning: divide by zero encountered in scalar divide
  err = 1 / np.sqrt(np.sum(w))
/Users/mdallas/Desktop/exovetter/exovetter/leo.py:16: RuntimeWarning: invalid value encountered in scalar divide
  mean =

,tce,num_transits,pri,phase_pri,sec,phase_sec,ter,phase_ter,pos,phase_pos,...,MES_series,MES,SHP,CHI,med_chases,mean_chases,max_SES,DMM,chases,rubble
0,TOI_1000.01,10,2,0.016144,137,1.105854,80,0.645754,218,1.759680,...,"[-0.2090148778842991, -0.23308397675005083, 0....",15.664582,0.155456,48.033949,1.0,1.000000,6.037704,0.975115,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.032539278039349, 0.9680008131755765, 0.9679..."
1,TOI_1001.01,12,1,0.014415,30,0.432458,0,0.000000,102,1.470358,...,"[0.5426588651499031, 0.8711703692165909, 1.169...",33.419850,0.083285,58.788465,1.0,1.000000,11.641219,0.973545,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.015407765756947, 0.9791389023182897, 0.9791..."
2,TOI_1004.01,13,1,0.010762,95,1.022399,254,2.733571,301,3.239389,...,"[0.36411016031087456, 0.3594709326814653, 0.53...",14.336500,0.158441,25.698710,1.0,0.877539,5.932953,0.920783,"[0.46429160831377186, 1.0, 1.0, 1.0, 1.0, 0.93...","[1.0190509273820185, 0.9705213873705859, 1.019..."
3,TOI_1007.01,4,1,0.017992,180,3.238575,343,6.315221,41,0.737675,...,"[-0.6947135502056714, -1.141752249511981, -1.2...",27.677785,0.081592,97.494419,1.0,1.000000,14.923815,0.990903,"[1.0, 1.0, 1.0, 1.0]","[1.0141570409181346, 1.014139870062947, 1.0141..."
4,TOI_1011.01,10,0,0.000000,156,1.401446,124,1.113970,31,0.278493,...,"[-1.2816162447661659, -1.5052944070157859, -1....",13.141436,0.187390,38.326712,1.0,1.000000,5.040320,1.007668,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.9879331591420123, 0.9298157913777899, 0.987..."


The following metrics are generated for each tce/light curve when using vet.run_all()

In [9]:
display(results.columns.to_list())

['tce',
 'num_transits',
 'pri',
 'phase_pri',
 'sec',
 'phase_sec',
 'ter',
 'phase_ter',
 'pos',
 'phase_pos',
 'sigma_pri',
 'sigma_sec',
 'sigma_ter',
 'sigma_pos',
 'false_alarm_threshold',
 'Fred',
 'raw_lpp',
 'norm_lpp',
 'oe_sigma',
 'odd_depth',
 'even_depth',
 'transit_phase_coverage',
 'msg',
 'amp',
 'sig_w',
 'sig_r',
 'err',
 'SES_series',
 'dep_series',
 'err_series',
 'MES_series',
 'MES',
 'SHP',
 'CHI',
 'med_chases',
 'mean_chases',
 'max_SES',
 'DMM',
 'chases',
 'rubble']

vet.run_all() can be used with verbose=True to print out which tce/light curve is being vetted and how long the vetting takes. 

In [10]:
utils.run_all(tces, lcs, verbose=True)

Vetting TOI_1000.01 :
VizTransits finished in 0.0026009082794189453 s.
ModShift finished in 0.0019757747650146484 s.
Lpp finished in 0.11938214302062988 s.
OddEven finished in 0.0006961822509765625 s.
TransitPhaseCoverage finished in 0.0002238750457763672 s.
Sweet finished in 0.0022652149200439453 s.


/Users/mdallas/Desktop/exovetter/exovetter/leo.py:16: RuntimeWarning: invalid value encountered in scalar divide
  mean = np.sum(w * y) / np.sum(w)
/Users/mdallas/Desktop/exovetter/exovetter/leo.py:21: RuntimeWarning: divide by zero encountered in scalar divide
  err = 1 / np.sqrt(np.sum(w))


LeoTransitEvents finished in 0.09229278564453125 s.

Vetting TOI_1001.01 :
VizTransits finished in 0.0017790794372558594 s.
ModShift finished in 0.0015358924865722656 s.
Lpp finished in 0.12367486953735352 s.
OddEven finished in 0.0008358955383300781 s.
TransitPhaseCoverage finished in 0.00024127960205078125 s.
Sweet finished in 0.0023260116577148438 s.


/Users/mdallas/Desktop/exovetter/exovetter/leo.py:16: RuntimeWarning: invalid value encountered in scalar divide
  mean = np.sum(w * y) / np.sum(w)
/Users/mdallas/Desktop/exovetter/exovetter/leo.py:21: RuntimeWarning: divide by zero encountered in scalar divide
  err = 1 / np.sqrt(np.sum(w))


LeoTransitEvents finished in 0.09924602508544922 s.

Vetting TOI_1004.01 :
VizTransits finished in 0.0021109580993652344 s.
ModShift finished in 0.0017731189727783203 s.
Lpp finished in 0.17325520515441895 s.
OddEven finished in 0.0008318424224853516 s.
TransitPhaseCoverage finished in 0.00026297569274902344 s.
Sweet finished in 0.0033299922943115234 s.


...

LeoTransitEvents finished in 0.2060708999633789 s.

Vetting TOI_1007.01 :
VizTransits finished in 0.0016508102416992188 s.
ModShift finished in 0.002147197723388672 s.
Lpp finished in 0.1242671012878418 s.
OddEven finished in 0.0006961822509765625 s.
TransitPhaseCoverage finished in 0.0002219676971435547 s.
Sweet finished in 0.002146005630493164 s.


/Users/mdallas/Desktop/exovetter/exovetter/leo.py:16: RuntimeWarning: invalid value encountered in scalar divide
  mean = np.sum(w * y) / np.sum(w)
/Users/mdallas/Desktop/exovetter/exovetter/leo.py:21: RuntimeWarning: divide by zero encountered in scalar divide
  err = 1 / np.sqrt(np.sum(w))


LeoTransitEvents finished in 0.09681200981140137 s.

Vetting TOI_1011.01 :
VizTransits finished in 0.0021882057189941406 s.
ModShift finished in 0.0017290115356445312 s.
Lpp finished in 0.1316239833831787 s.
OddEven finished in 0.0008869171142578125 s.
TransitPhaseCoverage finished in 0.000270843505859375 s.
Sweet finished in 0.0029489994049072266 s.
LeoTransitEvents finished in 0.10241985321044922 s.

Execution time: 1.3098130226135254 s


/Users/mdallas/Desktop/exovetter/exovetter/leo.py:16: RuntimeWarning: invalid value encountered in scalar divide
  mean = np.sum(w * y) / np.sum(w)
/Users/mdallas/Desktop/exovetter/exovetter/leo.py:21: RuntimeWarning: divide by zero encountered in scalar divide
  err = 1 / np.sqrt(np.sum(w))


,tce,num_transits,pri,phase_pri,sec,phase_sec,ter,phase_ter,pos,phase_pos,...,MES_series,MES,SHP,CHI,med_chases,mean_chases,max_SES,DMM,chases,rubble
0,TOI_1000.01,10,2,0.016144,137,1.105854,80,0.645754,218,1.759680,...,"[-0.2090148778842991, -0.23308397675005083, 0....",15.664582,0.155456,48.033949,1.0,1.000000,6.037704,0.975115,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.032539278039349, 0.9680008131755765, 0.9679..."
1,TOI_1001.01,12,1,0.014415,30,0.432458,0,0.000000,102,1.470358,...,"[0.5426588651499031, 0.8711703692165909, 1.169...",33.419850,0.083285,58.788465,1.0,1.000000,11.641219,0.973545,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.015407765756947, 0.9791389023182897, 0.9791..."
2,TOI_1004.01,13,1,0.010762,95,1.022399,254,2.733571,301,3.239389,...,"[0.36411016031087456, 0.3594709326814653, 0.53...",14.336500,0.158441,25.698710,1.0,0.877539,5.932953,0.920783,"[0.46429160831377186, 1.0, 1.0, 1.0, 1.0, 0.93...","[1.0190509273820185, 0.9705213873705859, 1.019..."
3,TOI_1007.01,4,1,0.017992,180,3.238575,343,6.315221,41,0.737675,...,"[-0.6947135502056714, -1.141752249511981, -1.2...",27.677785,0.081592,97.494419,1.0,1.000000,14.923815,0.990903,"[1.0, 1.0, 1.0, 1.0]","[1.0141570409181346, 1.014139870062947, 1.0141..."
4,TOI_1011.01,10,0,0.000000,156,1.401446,124,1.113970,31,0.278493,...,"[-1.2816162447661659, -1.5052944070157859, -1....",13.141436,0.187390,38.326712,1.0,1.000000,5.040320,1.007668,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.9879331591420123, 0.9298157913777899, 0.987..."


Run all can also be used with a plot argument to generate diagnostic plots. Setting plot=True will create a pdf for each tce/light curve in the current working directory, and adding a plot_dir argument will place them to a specified path as shown below.

In [11]:
plot_dir = os.getcwd() + '/run_all_plots/'
utils.run_all(tces, lcs, plot=True, plot_dir=plot_dir, verbose=True)

Vetting TOI_1000.01 :
VizTransits finished in 0.0023818016052246094 s.
ModShift finished in 0.20995092391967773 s.
Lpp finished in 0.2312309741973877 s.
OddEven finished in 0.06843924522399902 s.
TransitPhaseCoverage finished in 0.032601118087768555 s.
Sweet finished in 0.07028007507324219 s.


/Users/mdallas/Desktop/exovetter/exovetter/leo.py:16: RuntimeWarning: invalid value encountered in scalar divide
  mean = np.sum(w * y) / np.sum(w)
/Users/mdallas/Desktop/exovetter/exovetter/leo.py:21: RuntimeWarning: divide by zero encountered in scalar divide
  err = 1 / np.sqrt(np.sum(w))


LeoTransitEvents finished in 0.10457897186279297 s.

Vetting TOI_1001.01 :
VizTransits finished in 0.00249481201171875 s.
ModShift finished in 0.08408093452453613 s.
Lpp finished in 0.25842905044555664 s.
OddEven finished in 0.0720529556274414 s.
TransitPhaseCoverage finished in 0.040396928787231445 s.
Sweet finished in 0.09176993370056152 s.
LeoTransitEvents finished in 0.10706806182861328 s.



/Users/mdallas/Desktop/exovetter/exovetter/leo.py:16: RuntimeWarning: invalid value encountered in scalar divide
  mean = np.sum(w * y) / np.sum(w)
/Users/mdallas/Desktop/exovetter/exovetter/leo.py:21: RuntimeWarning: divide by zero encountered in scalar divide
  err = 1 / np.sqrt(np.sum(w))


Vetting TOI_1004.01 :
VizTransits finished in 0.0025000572204589844 s.
ModShift finished in 0.07913088798522949 s.
Lpp finished in 0.2831149101257324 s.
OddEven finished in 0.0731649398803711 s.
TransitPhaseCoverage finished in 0.034700870513916016 s.
Sweet finished in 0.09195399284362793 s.


...

/Users/mdallas/Desktop/exovetter/exovetter/leo.py:16: RuntimeWarning: invalid value encountered in scalar divide
  mean = np.sum(w * y) / np.sum(w)
/Users/mdallas/Desktop/exovetter/exovetter/leo.py:21: RuntimeWarning: divide by zero encountered in scalar divide
  err = 1 / np.sqrt(np.sum(w))


LeoTransitEvents finished in 0.09392428398132324 s.

Vetting TOI_1011.01 :
VizTransits finished in 0.0024030208587646484 s.
ModShift finished in 0.0794069766998291 s.
Lpp finished in 0.24875187873840332 s.
OddEven finished in 0.07429099082946777 s.
TransitPhaseCoverage finished in 0.03765296936035156 s.
Sweet finished in 0.08509302139282227 s.


/Users/mdallas/Desktop/exovetter/exovetter/leo.py:16: RuntimeWarning: invalid value encountered in scalar divide
  mean = np.sum(w * y) / np.sum(w)
/Users/mdallas/Desktop/exovetter/exovetter/leo.py:21: RuntimeWarning: divide by zero encountered in scalar divide
  err = 1 / np.sqrt(np.sum(w))


LeoTransitEvents finished in 0.0972909927368164 s.

Execution time: 7.906814813613892 s


,tce,num_transits,pri,phase_pri,sec,phase_sec,ter,phase_ter,pos,phase_pos,...,MES_series,MES,SHP,CHI,med_chases,mean_chases,max_SES,DMM,chases,rubble
0,TOI_1000.01,10,2,0.016144,137,1.105854,80,0.645754,218,1.759680,...,"[-0.2090148778842991, -0.23308397675005083, 0....",15.664582,0.155456,48.033949,1.0,1.000000,6.037704,0.975115,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.032539278039349, 0.9680008131755765, 0.9679..."
1,TOI_1001.01,12,1,0.014415,30,0.432458,0,0.000000,102,1.470358,...,"[0.5426588651499031, 0.8711703692165909, 1.169...",33.419850,0.083285,58.788465,1.0,1.000000,11.641219,0.973545,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.015407765756947, 0.9791389023182897, 0.9791..."
2,TOI_1004.01,13,1,0.010762,95,1.022399,254,2.733571,301,3.239389,...,"[0.36411016031087456, 0.3594709326814653, 0.53...",14.336500,0.158441,25.698710,1.0,0.877539,5.932953,0.920783,"[0.46429160831377186, 1.0, 1.0, 1.0, 1.0, 0.93...","[1.0190509273820185, 0.9705213873705859, 1.019..."
3,TOI_1007.01,4,1,0.017992,180,3.238575,343,6.315221,41,0.737675,...,"[-0.6947135502056714, -1.141752249511981, -1.2...",27.677785,0.081592,97.494419,1.0,1.000000,14.923815,0.990903,"[1.0, 1.0, 1.0, 1.0]","[1.0141570409181346, 1.014139870062947, 1.0141..."
4,TOI_1011.01,10,0,0.000000,156,1.401446,124,1.113970,31,0.278493,...,"[-1.2816162447661659, -1.5052944070157859, -1....",13.141436,0.187390,38.326712,1.0,1.000000,5.040320,1.007668,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.9879331591420123, 0.9298157913777899, 0.987..."


Lastly, in order to specify which vetters to use in case only a few are needed a list of the vetting objects with desired initialization settings can be supplied to the 'vetters' argument of vet.run_all(). For this example we use the default ModShift and SWEET vetters in addition to the OddEven vetter with the in-transit duration to use for depth calculation set to 0.9 rather than the defualt 0.3. 

In [12]:
utils.run_all(tces, lcs, vetters=[vet.ModShift(), vet.OddEven(dur_frac=0.9), vet.Sweet()], verbose=True)


Vetting TOI_1000.01 :
ModShift finished in 0.0028450489044189453 s.
OddEven finished in 0.0007598400115966797 s.
Sweet finished in 0.0027332305908203125 s.

Vetting TOI_1001.01 :
ModShift finished in 0.0018889904022216797 s.
OddEven finished in 0.0007419586181640625 s.
Sweet finished in 0.002672910690307617 s.

Vetting TOI_1004.01 :
ModShift finished in 0.002496004104614258 s.
OddEven finished in 0.0007910728454589844 s.
Sweet finished in 0.0038080215454101562 s.

Vetting TOI_1007.01 :
ModShift finished in 0.0017549991607666016 s.
OddEven finished in 0.0006771087646484375 s.
Sweet finished in 0.002209901809692383 s.

Vetting TOI_1011.01 :
ModShift finished in 0.0016150474548339844 s.
OddEven finished in 0.0006268024444580078 s.
Sweet finished in 0.002237081527709961 s.

Execution time: 0.02939009666442871 s


,tce,pri,phase_pri,sec,phase_sec,ter,phase_ter,pos,phase_pos,sigma_pri,sigma_sec,sigma_ter,sigma_pos,false_alarm_threshold,Fred,oe_sigma,odd_depth,even_depth,msg,amp
0,TOI_1000.01,2,0.016144,137,1.105854,80,0.645754,218,1.759680,-1168.2775953798825,-255.59248575896515,-130.1666626099337,253.535094767586,2.083895,87.132766,0.11708056156677722,"[0.9996212188055517, 0.00025291803112453874]","[0.9996630961936666, 0.00025291803112453874]",WARN: SWEET test finds signal at HALF transit ...,"[[3.799398207380298e-05, 9.934750089138508e-06..."
1,TOI_1001.01,1,0.014415,30,0.432458,0,0.000000,102,1.470358,-1568.8283223650108,-33.924238881232235,-1529.0422908228486,133.6096812847262,1.784431,65.986459,0.15922069679765224,"[0.9992924616861835, 0.000236883368255734]","[0.9992391221680839, 0.000236883368255734]",WARN: SWEET test finds signal at HALF transit ...,"[[5.1784117353467924e-05, 1.1265374479362451e-..."
2,TOI_1004.01,1,0.010762,95,1.022399,254,2.733571,301,3.239389,-1268.0088741167472,-336.7165672342121,-250.5419722961238,198.57846354330422,2.169563,67.124111,0.13104560767548495,"[0.9997420982958938, 0.0001808266391592816]","[0.9997756102596354, 0.0001808266391592816]",OK: SWEET finds no out-of-transit variability ...,"[[1.1341520393822437e-05, 6.101790144336315e-0..."
3,TOI_1007.01,1,0.017992,180,3.238575,343,6.315221,41,0.737675,-956.6987340000453,-75.74323674784375,-71.36631488478045,66.71551316854399,2.230979,47.185360,0.37564799384915315,"[0.9987126524059333, 0.00020984479862822596]","[0.9988241317181414, 0.00020984479862822596]",OK: SWEET finds no out-of-transit variability ...,"[[1.0404896455887515e-05, 9.859764607842088e-0..."
4,TOI_1011.01,0,0.000000,156,1.401446,124,1.113970,31,0.278493,-1019.4868240853754,-226.10012335337737,-184.899445642354,213.17233950043112,2.093797,72.902401,0.5799929193842958,"[0.9998050250254642, 0.00011082757983964489]","[0.9998959295582586, 0.00011082757983964489]",OK: SWEET finds no out-of-transit variability ...,"[[1.2134464387025161e-05, 4.865065235028228e-0..."
